In [ ]:
#import parameters 
from parameters import *

## Read the polygons file 

In [ ]:
import os 

#verify that the file existe 
print(os.path.isfile(polygon_file))

## Read the polygons file 

In [ ]:
#insert the polygons in a geopandas dataframe
import geopandas as gdp 

polygons = gdp.read_file(polygon_file)
polygons.head()

In [ ]:
#only select the first one (debug only)
polygons = polygons[polygons['id'] == 1]
polygons

In [ ]:
#transform the polygon into a square
from utils import *

squares = polygons
squares['geometry'] = squares['geometry'].map(to_square)

squares.head()

In [ ]:
#create a list of ee object based on the shapely figures in geopandas
import ee

ee.Initialize() 

#decide not to use a lis comprehension for readability 
ee_squares = []
for index, row in squares.iterrows():
    ee_square = ee.Geometry.Polygon(list(row['geometry'].exterior.coords))
    ee_squares.append(ee_square)
    
#add these ee_polygons to he dataset 
squares['ee_geometry'] = ee_squares
squares.head()

In [ ]:
# check if the result is effectively a square
from sepal_ui import mapping as sm 
import geemap

m = sm.SepalMap()

#displays the first nb_squares squares
for index, row in squares.iterrows():
    m.addLayer(row['ee_geometry'], {'color':'red'}, row['Name'])
    if row['id'] == 1:
        m.centerObject(row['ee_geometry'], zoom=14)
    if index > min(len(squares), nb_squares):
        break
m

## Create the tif file in earthengine 

In [ ]:
from sepal_ui import sepalwidgets as sw

output = sw.Alert()
output.show()

In [ ]:
from scripts import *

path = createPDF(polygon_file, squares, bands_combo, sources, output)
sw.DownloadBtn('clickable link', path)